In [14]:
from configparser import ConfigParser
from openai import OpenAI
config = ConfigParser()
config.read('../config.ini')
openai_api_key = config['LangChain']['openai_api_key']

client = OpenAI(api_key=openai_api_key)

In [2]:
import re

def extract_sql_query(response_text):
    """
    Extracts an SQL query from a given response text that contains the query wrapped within ```sql``` markers.
    
    Parameters:
    - response_text (str): The complete text returned by the model, possibly containing SQL and other descriptive text.
    
    Returns:
    - str: The extracted SQL query or a message indicating no query was found.
    """
    # Regular expression to find text enclosed in ```sql
    match = re.search(r"```sql\n([\s\S]*?)\n```", response_text)
    if match:
        return match.group(1)
    else:
        return "No SQL query found."

In [11]:
# Detailed descriptions of the database schema
schema_description = """
Given the following SQL tables, your job is to write queries given a user’s request. You must always put LIMIT at 20 except the users want to see more but the maximum LIMIT is 100 couldn't be more even the users asks for it.
- Orders: Contains records of customer orders. Fields include:
  - OrderID (int): Primary key, unique identifier for each order.
  - CustomerID (int): Foreign key to Customers table, identifies the customer who placed the order.
  - OrderDate (datetime): The date when the order was placed.
  - OrderTime (varchar): The time when the order was placed.
- OrderDetails: Contains details about items in each order. Fields include:
  - OrderDetailID (int): Primary key, unique identifier for each order detail entry.
  - OrderID (int): Foreign key to Orders table, identifies the order this detail belongs to.
  - ProductID (int): Foreign key to Products table, identifies the product ordered.
  - Quantity (int): The number of units of the product ordered.
- Products: Contains information about products. Fields include:
  - ProductID (int): Primary key, unique identifier for each product.
  - ProductName (varchar): The name of the product.
  - Category (varchar): The category to which the product belongs.
  - UnitPrice (decimal): The price per unit of the product.
  - Stock (int): The current stock level of the product.
- Customers: Contains information about customers. Fields include:
  - CustomerID (int): Primary key, unique identifier for each customer.
  - FirstName (varchar): The first name of the customer.
  - LastName (varchar): The last name of the customer.
  - Email (varchar): The email address of the customer.
  - Phone (varchar): The phone number of the customer.

"""


In [13]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'system','content': schema_description},
        {'role': 'user', 'content': 'Find the names and email addresses of customers who ordered more than 5 units of any product in March 2021? I want to see around 200 rows'}
    ]
)   

print((completion.choices[0].message.content))


```sql
SELECT c.FirstName, c.LastName, c.Email
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN OrderDetails od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
WHERE od.Quantity > 5
AND o.OrderDate >= '2021-03-01' AND o.OrderDate < '2021-04-01'
LIMIT 100;
```
